In [133]:
import numpy as np
import json
import pandas as pd

with open("processed_data/sports.json","r") as f:
    sports = json.load(f)

with open("processed_data/finalData2.json","r") as f:
    Data = json.load(f)

with open("processed_data/N0.json","r") as f:
    N0 = json.load(f)



In [134]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder

# 定义MLP模型
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(7, 5)  # 输入层到隐藏层1 (9个输入)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))  # 使用ReLU激活函数
        return x


In [135]:
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dataValue = Data.values()

total_data = list(itertools.chain(*dataValue))
total_data = np.array(total_data)
print(total_data.shape)

total_x = total_data[:,:7]
total_y = total_data[:,7]

label_encoder = LabelEncoder()
label_encoder.fit([-1, 0, 1, 2, 3])
total_y_encoded = label_encoder.transform(total_y)

X_train, X_test, y_train, y_test = train_test_split(total_x, total_y_encoded, test_size=0.3, random_state=42)


X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


model = MLPModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)



# 训练模型
epochs = 50  # 训练50个epoch
for epoch in range(epochs):
    model.train()  # 设置模型为训练模式

    # 清空梯度
    optimizer.zero_grad()

    # 前向传播
    outputs = model(X_train)

    # 计算损失
    loss = criterion(outputs, y_train)

    # 反向传播和优化
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 进行预测（示例）
model.eval()  # 设置模型为评估模式
with torch.no_grad():
    # 假设我们有一个测试样本
    y_pre = model(X_test)

    # 获取预测的类别
    _, predicted = torch.max(y_pre, 1)
    print(type(predicted))
    print(predicted)
    # predicted = predicted.numpy()
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(y_test)    
    print(f"Accuracy: {accuracy:.4f}")


(32176, 9)
Epoch [10/50], Loss: 1.5607
Epoch [20/50], Loss: 1.4909
Epoch [30/50], Loss: 1.4331
Epoch [40/50], Loss: 1.3882
Epoch [50/50], Loss: 1.3543
<class 'torch.Tensor'>
tensor([0, 1, 0,  ..., 0, 0, 0])
Accuracy: 0.7426
